In [ ]:
# Load imports
import os
import json
from rich import print
from cema.script.util import load_config, load_scenario
from cema.llm import ChatHandlerConfig, ChatHandlerFactory, verbalize

os.chdir("/home/balint/cema")

CARLA does not seem to be installed. CARLA-related functionality will not be available.
No module named 'carla'
INFO 02-11 16:21:38 __init__.py:190] Automatically detected platform cuda.


In [2]:
# Set configuations
scenario = 1
query_idx = 1
model = "llama-3B"

In [18]:
# Load scenario and query and observations
config = load_config(None, scenario)
agent, query = load_scenario(scenario, query_idx)

In [25]:
# Verbalize the road layout for the scenario
verbalized_scenario = verbalize.scenario(
    config,
    agent.scenario_map,
    agent.observations,
    query,
    add_rules=False,
    add_road_layout=False,
    f_subsample=2,
    control_signals=["position", "speed"])
verbalized_scenario += " Give a concise explanation."
print("Verbalized scenario:", verbalized_scenario)

Verbalized scenario: The scenario has 2 agents. Agent 0 is the autonomous vehicle (ego vehicle).

The ego vehicle observed the following control signals for each agent:

Agent 0:
  Position: [[-11.58, -1.75], [-10.59, -1.75], [ -9.59, -1.75], [ -8.6 , -1.75], [ -7.6 , -1.75], [ -6.61, -1.75], 
[ -5.61, -1.75], [ -4.61, -1.75], [ -3.62, -1.75], [ -2.62, -1.75], [ -1.62, -1.75], [ -0.62, -1.75], [ 0.37, 
-1.75], [ 1.36, -1.75], [ 2.32, -1.75], [ 3.25, -1.75], [ 4.15, -1.75], [ 5.03, -1.75], [ 5.88, -1.75], [ 6.72, 
-1.75], [ 7.54, -1.75], [ 8.36, -1.72], [ 9.18, -1.67], [ 10.01, -1.59], [ 10.84, -1.49], [ 11.67, -1.36], [ 12.5 ,
-1.21], [ 13.34, -1.04], [ 14.18, -0.85], [ 15.03, -0.64], [ 15.88, -0.43], [ 16.74, -0.21], [ 17.62, 0.02], [ 18.5
, 0.25], [ 19.38, 0.48], [ 20.28, 0.7 ], [ 21.18, 0.92], [ 22.09, 1.11], [ 23. , 1.29], [ 23.91, 1.44], [ 24.83, 
1.57]]
  Speed: [9.93, 9.94, 9.94, 9.95, 9.95, 9.96, 9.96, 9.97, 9.97, 9.97, 9.98, 9.98, 9.98, 9.81, 9.5 , 9.21, 8.95, 
8.72, 8.51, 8.32, 8.15, 8.22, 8.26, 8.3 , 8.36, 8.42, 8.49, 8.57, 8.65, 8.74, 8.83, 8.93, 9.03, 9.12, 9.19, 9.24, 
9.27, 9.29, 9.28, 9.25, 9.2 ]

Agent 1:
  Position: [[ 0.77, 1.75], [ 1.75, 1.72], [ 2.7 , 1.65], [ 3.64, 1.55], [ 4.56, 1.42], [ 5.47, 1.26], [ 6.37, 
1.08], [ 7.28, 0.88], [ 8.18, 0.66], [ 9.08, 0.44], [ 9.98, 0.21], [10.89, -0.03], [11.81, -0.27], [12.73, -0.51], 
[13.65, -0.74], [14.58, -0.95], [15.51, -1.15], [16.43, -1.33], [17.36, -1.47], [18.29, -1.59], [19.2 , -1.68], 
[20.12, -1.71], [21.02, -1.72], [21.92, -1.73], [22.84, -1.75], [23.76, -1.75], [24.69, -1.75], [25.63, -1.75], 
[26.57, -1.75], [27.51, -1.75], [28.43, -1.75], [29.34, -1.75], [30.22, -1.75], [31.08, -1.75], [31.91, -1.75], 
[32.71, -1.75], [33.48, -1.75], [34.21, -1.75], [34.91, -1.75], [35.58, -1.75], [36.21, -1.75]]
  Speed: [9.9 , 9.71, 9.5 , 9.37, 9.28, 9.24, 9.23, 9.25, 9.27, 9.31, 9.35, 9.41, 9.46, 9.5 , 9.52, 9.52, 9.49, 
9.44, 9.38, 9.3 , 9.2 , 9.09, 9.02, 9.08, 9.16, 9.24, 9.31, 9.38, 9.44, 9.36, 9.22, 9.03, 8.79, 8.51, 8.22, 7.91, 
7.59, 7.26, 6.92, 6.59, 6.27]


What if agent 1 had not changed lane right? Give a concise explanation.

In [6]:
# Load LLM interaction interface and scenario
configs = json.load(open("scenarios/llm/model_configs.json", "r", encoding="utf-8"))
config = configs["base"]
if model in configs:
    config.update(configs[model])
else:
    print("Model '%s' not found in model_configs.json", model)
config = ChatHandlerConfig(config)
chat_handler = ChatHandlerFactory.create_chat_handler(model, config)

Model llama-3B not found among handlers. Using LLMChatHandler.


INFO 02-11 16:27:26 config.py:542] This model supports multiple tasks: {'embed', 'generate', 'classify', 'reward', 'score'}. Defaulting to 'generate'.
INFO 02-11 16:27:26 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llama/Llama-3.

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-11 16:27:35 model_runner.py:1115] Loading model weights took 5.9848 GB
INFO 02-11 16:28:09 worker.py:267] Memory profiling takes 32.65 seconds
INFO 02-11 16:28:09 worker.py:267] the current vLLM instance can use total_gpu_memory (11.99GiB) x gpu_memory_utilization (0.90) = 10.79GiB
INFO 02-11 16:28:09 worker.py:267] model weights take 5.98GiB; non_torch_memory takes -1.26GiB; PyTorch activation peak memory takes 1.26GiB; the rest of the memory reserved for KV Cache is 4.81GiB.
INFO 02-11 16:28:10 executor_base.py:110] # CUDA blocks: 2815, # CPU blocks: 2340
INFO 02-11 16:28:10 executor_base.py:115] Maximum concurrency for 16384 tokens per request: 2.75x
INFO 02-11 16:28:11 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████| 35/35 [01:36<00:00,  2.75s/it]

INFO 02-11 16:29:47 model_runner.py:1562] Graph capturing finished in 97 secs, took 0.00 GiB
INFO 02-11 16:29:47 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 132.54 seconds


In [26]:
from vllm import SamplingParams
chat_handler.reset_history()
chat_handler._sampling_params = SamplingParams(max_tokens=256, temperature=0.6)
response, _ = chat_handler.interact(verbalized_scenario)

In [27]:
print(response[0])

If Agent 1 had not changed lane to the right at the current position and speed, the autonomous vehicle (Agent 0) 
would need to continue following the same trajectory.

Explanation:
Agent 1's current position and speed are closer to the same as Agent 0. If Agent 1 had not changed lane, it would 
have continued on the same path and would likely be at the same position and speed as Agent 0. The autonomous 
vehicle (Agent 0) would continue to follow Agent 1's trajectory, tracking its position and speed. This means the 
autonomous vehicle (Agent 0) would not need to adjust its lane or speed to avoid a collision, as it would be 
following Agent 1's unchanged trajectory.